In [1]:
import pandas as pd

df=pd.read_excel('Processed_Images_DataK18.xlsx')
fdf=df[(df['area'] >= 100) & (df['area'] <= 3000)].copy()
fdf.to_excel('Processed_Images_Data18_filtered.xlsx', index=False)

In [6]:
import pandas as pd

# Load your Excel sheet into a DataFrame
df = pd.read_excel('Processed_Images_Data21K_filtered_clustered.xlsx')

# Filter the DataFrame to contain only rows where 'classification' is 0
filtered_df = df[df['lympho_classification'] == 0]

# Display the filtered DataFrame
filtered_df.to_excel('final21K_forCC.xlsx', index=False)


In [1]:
import pandas as pd
import ast  # For safely evaluating strings containing lists

# Load the DataFrame
file_path = 'Processed_Images_Data18_filtered_classification_results.xlsx'  # Update this path to your actual file
df = pd.read_excel(file_path)

# Define a function to count lymphocyte neighbors and calculate their frequencies
def calculate_neighbor_frequencies(row, df):
    # Extract neighbor IDs from the cellularity string
    cellularity_dict = ast.literal_eval(row['cellularity'])
    neighbor_ids = cellularity_dict['neighbor_names']
    
    # Filter the DataFrame to include only rows with the same image_id and matching neighbor_ids
    neighbor_rows = df[(df['image_id'] == row['image_id']) & (df['nuclei_id'].isin(neighbor_ids))]
    lymphocyte_neighbors = neighbor_rows[neighbor_rows['lympho_classification'] == 1]

    # Calculate frequencies
    total_neighbors = len(neighbor_ids)
    lymphocyte_count = len(lymphocyte_neighbors)
    lymph_neighbor_frequency = lymphocyte_count / total_neighbors if total_neighbors > 0 else 0
    other_neighbor_frequency = (total_neighbors - lymphocyte_count) / total_neighbors if total_neighbors > 0 else 0

    return lymph_neighbor_frequency, other_neighbor_frequency

# Apply the function to each row to create the new columns
df['lymph_neighbor_frequency'], df['other_neighbor_frequency'] = zip(*df.apply(lambda row: calculate_neighbor_frequencies(row, df), axis=1))
df_final = df[['image_id', 'nuclei_id', 'lympho_classification', 'lymph_neighbor_frequency', 'other_neighbor_frequency']]

# Save the updated DataFrame to a new Excel file
output_file_path = 'final_merged18_classification_with_neighbors.xlsx'
df_final.to_excel(output_file_path, index=False)

# Display the updated DataFrame
print(df.head())


KeyboardInterrupt: 